In this notebook we find positions with significantly low/high MFE. 
First, we need to take the mfe per window scores that we have from "mfe_windows.ipynb" and map them to positions in the CDS. 
This is elaboratly explained in the description of the function "calc_mfe_per_position". 


## Imports

In [1]:
import os
from os import listdir
import numpy as np
import pandas as pd
import pickle
from Bio import AlignIO
import concurrent.futures
import statsmodels.stats.multitest
import scipy.stats
from typing import Tuple


## Functions

In [2]:
def nansumwrapper(vectors_to_sum, axis:int) -> float:
    '''
    We need to sum the columns of "mfe_windows_padded" and its shifted versions. We need for
    nan + nan to be equal nan but the numpy function np.nansum returns zero in that case. 
    (This is bad for us because a 0 score means positions with weak folding and this is not true for
    nan positions). So, we create a wrapper for that function, that returns a nan if all the values
    summed are equal nan.
    '''
    nan_positions = np.isnan(vectors_to_sum).all(axis) #positions where both vectors that are being summed are equal nan. 
    res = np.nansum(vectors_to_sum, axis) #the result of the summation, there are zeros where only nans are summed
    res[nan_positions] = np.nan #insert nans in those locations
    return(res)



In [3]:
def calc_mfe_per_position(mfe_windows:np.ndarray, data:str, padding_amount:int, window_size:int) -> np.ndarray:
    '''
    In this function we perform the transition from mfe of windows to mfe of positions in the cds. The mfe score of a position 
    is the mean of the scores of all windows the position is in. We are creating "window_size - 1" shifted versions of the mfe_window
    results vector and so we take the mean of the windows in a vectorized manner. 

    example: 
    let's say we have an original sequence of length 10, and the window size is 4. Then the number of windows is 7
    (length_cds - window_size + 1). 
    let's say the window scores are these: mfe_windows = [-1,-3,-2,-2,-4,-3,-2]
    according to the rule, the results should be:
    mfe_position_0 = mean(mfe_windows[0]) = -1 (position 0 is only in window 0)
    mfe_position_1 = mean(mfe_windows[0,1]) = ((-1-3)/2) = -2
    mfe_position_2 = mean(mfe_windows[0,1,2]) = ((-1-3-2)/3) = -2
    mfe_position_3 = mean(mfe_windows[0,1,2,3]) = ((-1-3-2-2)/4) = -2
    mfe_position_4 = mean(mfe_windows[1,2,3,4]) = ((-3-2-2-4)/4) = -2.75
    mfe_position_5 = mean((mfe_windows[2,3,4,5])) = ((-2-2-4-3)/4) = -2.75
    mfe_position_6 = mean((mfe_windows[3,4,5,6])) = ((-2-4-3-2)/4) = -2.75
    mfe_position_7 = mean((mfe_windows[4,5,6])) = ((-4,-3,-2)/3) = -3
    mfe_position_8 = mean((mfe_windows[5,6])) = ((-3,-2)/2) = -2.5
    mfe_position_9 = mean((mfe_windows[6])) = -2

    let's show that we are getting the same result using our method (but fast...)
    pad the mfe_windows vector: add nans to the end to make it in the length of "num_positions"
    mfe_windows = [-1,-3,-2,-2,-4,-3,-2] (length 7) -> mfe_windows_padded = [-1,-3,-2,-2,-4,-3,-2,nan,nan,nan] (length 10)
    shifted_1 = [nan,-1,-3,-2,-2,-4,-3,-2,nan,nan] (one score was moved from the end to the beginning)
    shifted_2 = [nan,nan,-1,-3,-2,-2,-4,-3,-2,nan] (two scores were moved from the end to the beginning)
    shifted_3 = [nan,nan,nan,-1,-3,-2,-2,-4,-3,-2] 
    (how many shifted versions do we need? window_size -1)

    now, we are summing mfe_windows,shifted_1,shifted_2 and shifted_3. 
    (in the code we do this one after the other so we dont have to keep all the matrices, because this is memory inefficient)
    We take the average of each column:
    mean_col0 = mean([-1,nan,nan,nan]) = -1
    mean_col1 = mean([-3,-1,nan,nan]) = -2
    mean_col2 = mean([-2,-3,-1,nan]) = -2
    mean_col3 = mean([-2,-2,-3,-1]) = -2
    mean_col4 = mean([-4,-2,-2,-3]) = -2.75
    mean_col5 = mean([-3,-4,-2,-2]) = -2.75
    mean_col6 = mean([-2,-3,-4,-2]) = -2.75
    mean_col7 = mean([nan,-2,-3,-4]) = -3
    mean_col8 = mean([nan,nan,-2,-3]) = -2.5
    mean_col9 = mean([nan,nan,nan,-2]) = -2
    (Notice that we are summing over the not-nan positions so we have to keep track of the amount of nans per position
    to calculate the mean). 

    we can see that we got the same results! but the second way can be vectorized so we will use it.
    '''
    #perform the padding
    if data == 'original':
        mfe_windows_padded = np.pad(mfe_windows, ((0,0),(0, padding_amount)), 'constant', constant_values=(np.nan))
    else:
        mfe_windows_padded = np.pad(mfe_windows, ((0,0),(0,0),(0, padding_amount)), 'constant', constant_values=(np.nan))
    #perform the averaging. #each time, we shift and sum. later we devide by the number of windows the position was in. 
    mfe_per_position = mfe_windows_padded.copy() #initialize
    nans_per_position = np.isnan(mfe_windows_padded).astype(int) #keep track of nans in order to devide later by num of non-nan positions
    axis = 1 if data == 'original' else 2
    for i in range(1,window_size):
        shifted = np.roll(mfe_windows_padded, i, axis = axis) #create the next shifted version
        nans_per_position = nans_per_position + np.isnan(shifted).astype(int) #where are the nans in this shifted version? 
        mfe_per_position = nansumwrapper([mfe_per_position, shifted], axis = 0)#sum the prior columns of the prior result with this shifted version

    not_nans_per_position = window_size - nans_per_position #for each column, how many elements were not equal to nan?
    mfe_per_position = mfe_per_position/not_nans_per_position #for each column, devide by the number of not nan elements
    return(mfe_per_position)

In [5]:
def trim_mfe_tails(mfe_scores_cur_ortholog_orig: np.ndarray, mfe_scores_cur_ortholog_col: np.ndarray, mfe_scores_cur_ortholog_ver:np.ndarray, cur_ortholog_length:int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''
    Our mfe scores are in shape (num_orthologs, max_num_positions) where max_num_positions is the number of positions of
    the longest ortholog, meaning the score vectors of the other orthologs have a nan "tail". We now want to remove that nan
    tail so the number of mfe scores of an ortholog will be equal to the amount of positions of the cds (at the next step,
    we will take these scores and align them to the MSA, meaning, insert deletions between the scores according to the MSA.)

    This funciton receives:
    "mfe_scores_cur_ortholog_orig" : the true mfe scores per position of a single ortholog (1D numpy array in shape (1, max_num_positions))
    "mfe_scores_cur_ortholog_col" : the mfe scores per position of the column randomization of a single ortholog (2D numpy array in shape (num_randomizations,max_num_positions))
    "mfe_scores_cur_ortholog_ver" : the mfe scores per position of the vertical randomization of a single ortholog (2D numpy array in shape (num_randomizations,max_num_positions))
    "cur_ortholog_length" : the length of the cds of this ortholog
    and returns:
    "mfe_scores_cur_ortholog_orig_trimmed": the true mfe scores per position of a single ortholog, removed of the nan tail (1D numpy array in shape (1,cur_ortholog_length))
    "mfe_scores_cur_ortholog_col_trimmed": the mfe scores per position of the column randomizations of a single ortholog, removed of the nan tail (2D numpy array in shape (num_randomizations,cur_ortholog_length))
    "mfe_scores_cur_ortholog_ver_trimmed": the mfe scores per position of the vertical randomizations of a single ortholog, removed of the nan tail (2D numpy array in shape (num_randomizations,cur_ortholog_length))
    '''
    indices_to_delete = np.arange(cur_ortholog_length,len(mfe_scores_cur_ortholog_orig)) #all indices larger than the length of the cur_ortholog
    assert(np.all(np.isnan(mfe_scores_cur_ortholog_orig[indices_to_delete]) == True)) #make sure we are deleting only nans and not a true mfe score  
    #delete for original:
    mfe_scores_cur_ortholog_orig = np.delete(mfe_scores_cur_ortholog_orig,indices_to_delete) #deleting these indices
    #delete for column rands:
    mfe_scores_cur_ortholog_col = np.delete(mfe_scores_cur_ortholog_col, indices_to_delete, axis = 1) 
    #delete for ver rands:
    mfe_scores_cur_ortholog_ver = np.delete(mfe_scores_cur_ortholog_ver, indices_to_delete, axis = 1) 
    return(mfe_scores_cur_ortholog_orig,mfe_scores_cur_ortholog_col,mfe_scores_cur_ortholog_ver)


In [7]:
def save_mfe_scores(gene:str, mean_mfe_orig:np.ndarray, mean_mfe_col:np.ndarray, mean_mfe_ver:np.ndarray) -> None:
    ''' 
    Save the scores to a the results folder
    '''
    output_path = f"../Results//mfe_scores/original/{gene}.pickle"
    with open(output_path, 'wb') as handle:
        pickle.dump(mean_mfe_orig, handle)
    !gzip $output_path
    
    output_path = f"../Results//mfe_scores/column/{gene}.pickle"
    with open(output_path, 'wb') as handle:
        pickle.dump(mean_mfe_col, handle)
    !gzip $output_path
        
    output_path = f"../Results//mfe_scores/vertical/{gene}.pickle"
    with open(output_path, 'wb') as handle:
        pickle.dump(mean_mfe_ver, handle)
    !gzip $output_path


In [8]:
def get_scores(original_scores: np.ndarray, random_scores: np.ndarray, good_mask: np.ndarray) -> pd.DataFrame:
    ''' 
    This functions returns the z-scores and respective p-values for each position. 
    '''
    #get the parameters of the normal distribution
    miu = np.mean(random_scores, axis = 0)
    sigma = np.std(random_scores, axis = 0)
    
    #get the z-scores
    z_scores = (original_scores - miu) / sigma
    
    #get the one-sided p-values 
    p_vals = scipy.stats.norm.sf(abs(z_scores))
    # correct FDR. we have nan values (for positions where the randomizations are the same and so sigma = 0) and the FDR correction function doesnt work with nans, 
    # so we will use a mask
    mask = np.isfinite(p_vals) #mask contains "True" only the non-nan positions
    pval_corrected = np.empty(p_vals.shape) #initilize the final result in the right dimensions
    pval_corrected.fill(np.nan) #fill it with nans
    pval_corrected[mask] = statsmodels.stats.multitest.multipletests(p_vals[mask],method='fdr_bh')[1] #insert the corrected p-vals at the non-nan positions
    
    res_df = pd.DataFrame(columns = ["z-score", "p-value", "corrected p-value", "good_position"])
    res_df.index.name='position of CDS'
    res_df["z-score"] = z_scores
    res_df["p-value"] = p_vals
    res_df["corrected p-value"] = pval_corrected
    res_df["good_position"] = good_mask

    return(res_df)  

In [10]:
def whole_pipline_single_gene(gene:str,window_size:int = 39) -> None:
    ''' 
    This function gets the mfe z-scores per CDS position of a single gene
    '''
    #try:
        
    ################# (1) window mfe scores to position mfe scores ################################################
    '''Download the mfe per window results'''
    mfe_orig = pd.read_pickle(f"../Results/window_mfe_scores/original/{gene}.pickle.gz")
    mfe_col = pd.read_pickle(f"../Results/window_mfe_scores/column/{gene}.pickle.gz")
    mfe_ver = pd.read_pickle(f"../Results/window_mfe_scores/vertical/{gene}.pickle.gz")
    '''Define needed parameters'''
    num_randomizations,num_orthologs, max_num_windows  = mfe_col.shape #get the dimentions of the results of the gene
    max_num_positions = max_num_windows + window_size -1 #the num of cds positions (not including the stop codon) of the ortholog with the longest cds
    pad = max_num_positions -  max_num_windows #the needed amount of nucleotides to pad the input of the moving average function to be the size of the output

    '''From mfe per window to mfe per position, for the original and randomized CDSs'''
    mfe_per_position_orig = calc_mfe_per_position(mfe_orig, data = 'original', padding_amount = pad, window_size = window_size)
    del mfe_orig
    #print("done calculating mfe per position for orig")
    mfe_per_position_col = calc_mfe_per_position(mfe_col, data = 'column', padding_amount = pad, window_size = window_size)
    #print("done calculating mfe per position for col")
    del mfe_col
    mfe_per_position_ver = calc_mfe_per_position(mfe_ver, data = 'vertical', padding_amount = pad, window_size = window_size)
    #print("done calculating mfe per position for ver")
    del mfe_ver

    ################# (2) align to MSAs ################################################
    '''
    At this point we have our mfe scores per cds position, but the positions are not aligned to the MSA. 
    We have to align to the MSA in order to compare the mfe scores in each position. 
    We need to insert deletions where they occur in the MSA.
    We can get the deletion positions from the original MSA (the deletion positions are the same for the randomizations and the original)
    '''

    '''download the MSA in nucleotides'''
    path_NT = f"../co_trans_data/orthologs/nt_after_msa/{gene}.fasta.gz" #MSA in nucleotides
    local_path = f"./{gene}.fasta"
    !gzip -c -d $path_NT > $local_path #unzip without erasing the zipped version
    alignment_NT = AlignIO.read(local_path, "fasta")
    os.remove(local_path)
    '''Find all positions of deletions. Use a mask to place the mfe scores in the non-deletions positions.'''
    nuc_array = np.array(alignment_NT).astype('object') # an array where each column is a nucleotide position and each row is a cds of an ortholog
    non_deletions_mask = nuc_array != '-' #Get all locations of non-deletions in the MSA. 
    human_del_locs = np.where(~non_deletions_mask[0,:])[0] #we need the positions of the human ortholog deletions later on

    nuc_array_col = np.tile(nuc_array,(num_randomizations,1,1)) #duplicating "nuc_array" "num_randomizations" times for our column randomizations. We care only about the deletions positions here. 
    nuc_array_ver = np.tile(nuc_array,(num_randomizations,1,1)) #duplicating "nuc_array" "num_randomizations" times for our vertical randomizations. We care only about the deletions positions here. 

    '''Now we will iterate over the sequences of the original and random MSAs. For each sequence, we have an mfe scores
    per position vector in the size of (1, max_num_positions). For each sequence we will take the
    mfe results and we place them at the non-deletion locations according to our mask. For example, if the current ortholog has 
    12 positions and the longest one has 15 positions, the mfe_scores of our ortholog could look like
    [-1,-2,-1,-3,-6,-3,-6,-2,-2,-1,-1,-5,nan,nan,nan] -> 12 scores, 3 nans. Our original ortholog looks like : "ATGAAGCCTACC"
    The MSA could look like: "ATGAAGCCT---ACC---" -> 12 non-deletion positions (has to be the same as the number of scores in our vector)
    Then, we want to take our results, discard the nan tail and align them to the MSA. "plant" the deletions among them so the
    final result looks like this : [-1,-2,-1,-3,-6,-3,-6,-2,-2,-,-,-,-1,-1,-5,-,-,-]
    '''

    lengths_dict = pd.read_pickle(f"../co_trans_data/cds_lengths/{gene}.pickle") #a dictionary containing the lengths of the cdss

    #trim nan tails & insert deletions to align to the msa
    for cur_ortholog in range(num_orthologs):
        trimmed_orig, trimmed_col, trimmed_ver = trim_mfe_tails(mfe_per_position_orig[cur_ortholog,:],mfe_per_position_col[:,cur_ortholog,:],mfe_per_position_ver[:,cur_ortholog,:],lengths_dict[cur_ortholog])
        cur_mask = non_deletions_mask[cur_ortholog,:] #get positions of non deletions of this ortholog in the MSA 
        #inserting the mfe results to the non-deletion positions in the MSA:
        nuc_array[cur_ortholog,cur_mask] = trimmed_orig #original
        nuc_array_col[:,cur_ortholog,cur_mask] = trimmed_col #column rands
        nuc_array_ver[:,cur_ortholog,cur_mask] = trimmed_ver #vertical rands
    #print("done aligning to MSA")
    ################# (3) get average mfe per position ################################################
    '''Average over orthologs to get the mean mfe per position, for both original and randomized data.''' 
    '''remove deletion positions in the human ortholog (we want the final scores to regard only positions that exist
    in the human ortholog)'''
    nuc_array = np.delete(nuc_array, human_del_locs, axis = 1) #nuc_array is a 2D matrix holding the MSA in nucleotides
    nuc_array_col = np.delete(nuc_array_col, human_del_locs, axis = 2) 
    nuc_array_ver = np.delete(nuc_array_ver, human_del_locs, axis = 2) 

    '''change deletions (all other orthologs other than the human one still could have deletions) to nans for mean computation'''
    nuc_array[nuc_array == '-'], nuc_array_col[nuc_array_col == '-'], nuc_array_ver[nuc_array_ver == '-'] = np.nan, np.nan, np.nan  

    '''change type of data to float'''
    nuc_array = nuc_array.astype(float) #now that we dont have deletions we can change datatype to float and find nan locations
    nuc_array_col = nuc_array_col.astype(float)
    nuc_array_ver = nuc_array_ver.astype(float)

    '''calculate the mean of each position'''
    mean_mfe_orig, mean_mfe_col, mean_mfe_ver  = np.nanmean(nuc_array,axis = 0), np.nanmean(nuc_array_col,axis = 1), np.nanmean(nuc_array_ver,axis = 1) 

    '''there are positions that are found significant because of rounding issues-'''
    mean_mfe_orig, mean_mfe_col, mean_mfe_ver = np.round(mean_mfe_orig,decimals=8), np.round(mean_mfe_col,decimals=8), np.round(mean_mfe_ver,decimals=8)

    save_mfe_scores(gene, mean_mfe_orig, mean_mfe_col, mean_mfe_ver)
    #print("done saving the mfe scores")
    ################# (4) find significantly weak and strong positions ################################################
    '''First we will create a mask of valid positions:
    -Find positions with more than "allowed_nans" deletions (among orthologs) and do not use them (unreliable data - even if they are 
    found as significant- ignore them). We do not want to delete them because we want to keep the current length of the human cds.''' 
    percent_nans = np.sum(np.isnan(nuc_array),axis = 0) / num_orthologs
    del nuc_array, nuc_array_col, nuc_array_ver
    allowed_nans = 0.5
    good_positions_mask1 = percent_nans < allowed_nans # a "good position" is defined as a position where less than "allowed nans" of orthologs had a deletion

    '''we also do not want to consider positions with STD == 0 between the randomizations (or practically zero), meaning where all the randomizations are the same. 
    They could all be slightly bigger/smaller than the randomization because of rounding when averaging and result in a "significant" 
    positions without a real reason.'''
    good_positions_mask_2v = np.round(np.std(mean_mfe_ver,axis = 0),5) != 0   #find positions where the variation between the randomizations is larger than 0.00005
    good_positions_mask_2c = np.round(np.std(mean_mfe_col,axis = 0),5) != 0   #find positions where the variation between the randomizations is larger than 0.00005

    '''Combine both masks to obtain valid positions'''
    good_positions_mask_v = good_positions_mask1 & good_positions_mask_2v
    good_positions_mask_c = good_positions_mask1 & good_positions_mask_2c

    ''' Get the z-score and p-value for each position '''
    res_ver = get_scores(mean_mfe_orig, mean_mfe_ver, good_positions_mask_v) 
    res_col = get_scores(mean_mfe_orig, mean_mfe_col, good_positions_mask_c)

    '''save scores:'''
    output_path = f"../Results/z-scores/{gene}_column.pickle"
    with open(output_path, 'wb') as handle:
        pickle.dump(res_col, handle)
    output_path = f"../Results/z-scores/{gene}_vertical.pickle"
    with open(output_path, 'wb') as handle:
        pickle.dump(res_ver, handle)
        
#     except Exception as e:
#         file_object = open('../Results/AllGenes/mfe/errors_mfe_positions.txt', 'a')
#         file_object.write(f"gene {gene} failed with error: {e}")
#         file_object.close()



## Main

In [ ]:
#get genes to run on
mypath = "../Results/window_mfe_scores/original/" #all the genes with both column and vertical window scores
l=os.listdir(mypath)
genes_list=[x.split('.')[0] for x in l] #remove the extension

In [ ]:
#split genes to batchs to improve preformance

'''Create batches of genes'''
num_wanted_cpus = 30
num_genes = len(genes_list)
num_genes_per_batch = int(np.round(num_genes /num_wanted_cpus))
batches_of_genes = [genes_list[x:x+num_genes_per_batch] for x in range(0, num_genes, num_genes_per_batch)]



In [12]:
def do_for_single_batch(single_batch_genes: list) -> None:
    '''
    This function calls our main function "whole_pipline_single_gene" for batch of genes
    '''
    for gene in single_batch_genes:
        whole_pipline_single_gene(gene)
        

In [ ]:
#run in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:
    futures = []
    for single_batch in batches_of_genes:
        futures.append(executor.submit(do_for_single_batch, single_batch_genes = single_batch))
                      